In [ ]:
#Feuture Selection

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore, ks_2samp
import seaborn as sns

%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Load the CSV file into a DataFrame
df = pd.read_csv(r'C:\Users\מיכאל\Desktop\air bnb germany project\df_level3final.csv', encoding='latin1',low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169001 entries, 0 to 169000
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Host Response Time        169001 non-null  float64
 1   Host Response Rate        169001 non-null  float64
 2   Is Superhost              169001 non-null  float64
 3   neighbourhood             169001 non-null  float64
 4   Postal Code               169001 non-null  float64
 5   Is Exact Location         169001 non-null  float64
 6   Property Type             169001 non-null  float64
 7   Accomodates               169001 non-null  float64
 8   Bathrooms                 169001 non-null  float64
 9   Bedrooms                  169001 non-null  float64
 10  Beds                      169001 non-null  float64
 11  Price                     169001 non-null  float64
 12  Guests Included           169001 non-null  float64
 13  Min Nights                169001 non-null  f

In [8]:
y=df['Price']
X = df.drop(columns=['Price'])

In [9]:
# Multivariable Analysis
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC


In [10]:
### Summarization and Selection of Variables 
# Fit models and determine if a feature is selected (1) or not (0)
lasso = Lasso(alpha=0.01).fit(X, y)
lasso_selected = (np.abs(lasso.coef_) > 0).astype(int)

svm = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
svm_selected = (np.abs(svm.coef_[0]) > 0).astype(int)

gb = GradientBoostingClassifier().fit(X, y)
gb_selected = (gb.feature_importances_ > 0).astype(int)

rf = RandomForestClassifier().fit(X, y)
rf_selected = (rf.feature_importances_ > 0).astype(int)

# Create a DataFrame to store results
selection_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso': lasso_selected,
    'SVM': svm_selected,
    'GradientBoost': gb_selected,
    'RandomForest': rf_selected
})

# Sum the number of selections for each feature
selection_df['Sum'] = selection_df[['Lasso', 'SVM', 'GradientBoost', 'RandomForest']].sum(axis=1)

# Output the results
print(selection_df)

                     Feature  Lasso  SVM  GradientBoost  RandomForest  Sum
0         Host Response Time      1    0              1             1    3
1         Host Response Rate      0    1              0             0    1
2               Is Superhost      1    0              1             1    3
3              neighbourhood      1    0              1             1    3
4                Postal Code      1    1              1             1    4
5          Is Exact Location      1    0              1             1    3
6              Property Type      1    1              1             1    4
7                Accomodates      1    1              1             1    4
8                  Bathrooms      0    0              0             0    0
9                   Bedrooms      0    0              0             0    0
10                      Beds      1    1              1             1    4
11           Guests Included      1    0              1             1    3
12                Min Nig

In [12]:
# Creating DataFrame with most valuable variables
 #Selecting variables with a sum of selections >= 3
final_var = selection_df[selection_df['Sum'] >= 3]['Feature'].tolist()
df_model = df[final_var].copy()
df_model['Price'] = df['Price'].copy()

# Output the result to verify
df_model.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169001 entries, 0 to 169000
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Host Response Time      169001 non-null  float64
 1   Is Superhost            169001 non-null  float64
 2   neighbourhood           169001 non-null  float64
 3   Postal Code             169001 non-null  float64
 4   Is Exact Location       169001 non-null  float64
 5   Property Type           169001 non-null  float64
 6   Accomodates             169001 non-null  float64
 7   Beds                    169001 non-null  float64
 8   Guests Included         169001 non-null  float64
 9   Min Nights              169001 non-null  float64
 10  Reviews                 169001 non-null  float64
 11  Overall Rating          169001 non-null  float64
 12  Cleanliness Rating      169001 non-null  float64
 13  Location Rating         169001 non-null  float64
 14  Value Rating        

In [13]:
# save the dataframe 
df_model.to_csv(r'C:\Users\מיכאל\Desktop\air bnb germany project\df_level4_feature_selection.csv')
print("saved")